## 1. What is a socket?

Think of a **socket** as a two‑way pipe between programs. At the OS level it’s just a file descriptor, so you read/write bytes the same way you do with a file.  Identified by **(IP address, port number, protocol)** on each end.

Why it matters:
* Every higher‑level protocol—HTTP, FTP, gRPC—rides on sockets.
* If a request “hangs,” knowing socket states helps you debug (SYN‑SENT, ESTABLISHED…).

```python
import socket
# create TCP client socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('example.com', 80))
sock.sendall(b'HEAD / HTTP/1.0\r\nHost: example.com\r\n\r\n')
print(sock.recv(40))
sock.close()
```

### Quick check

1. True / False Writing to a socket uses the same `send` sys‑call Linux uses for files.

2. A socket endpoint is uniquely identified by:
  a. port only   b. IP+port+protocol

<details><summary>Answer key</summary>

1. **True** – under the hood it’s `write()` on fd.
2. **b**.

</details>

## 2. IPv4 vs. IPv6 basics

*IPv4* uses 32‑bit dotted quad (e.g., `192.0.2.1`). Limited ~4 billion addresses.
*IPv6* uses 128‑bit hex blocks (`2001:db8::1`). Built‑in ideas: link‑local (`fe80::/10`), loopback `::1`, and huge address space.

Python constants: `socket.AF_INET` vs. `AF_INET6` when creating sockets.

```python
import socket
ipv4 = socket.getaddrinfo('localhost', 80, family=socket.AF_INET)
ipv6 = socket.getaddrinfo('localhost', 80, family=socket.AF_INET6)
print('v4', ipv4[0][4][0], 'v6', ipv6[0][4][0])
```

### Quick check

1. Loopback address in IPv6 is:
  a. 127.0.0.1   b. ::1

2. True / False IPv6 addresses can embed IPv4 in last 32 bits.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – called IPv4‑mapped.

</details>

## 3. TCP vs. UDP

**TCP** – connection‑oriented, reliable, ordered bytes. Cost: handshake latency, state.
**UDP** – connectionless datagrams, best‑effort, may drop or reorder packets. Useful for DNS, video, gaming where latency beats reliability.

```python
import socket
# UDP echo
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.sendto(b'ping', ('8.8.8.8', 53))  # just fires, no connect
```

### Quick check

1. Which guarantees in‑order delivery?
  a. TCP   b. UDP

2. True / False UDP requires handshake.

<details><summary>Answer key</summary>

1. **a**.
2. **False**.

</details>

## 4. Three‑way handshake & listen queue

Server `listen()` creates a *backlog queue*. TCP connect flow:
1. **SYN** client→server
2. **SYN‑ACK** server→client
3. **ACK** client→server → connection ESTABLISHED.

If backlog full, new SYNs are dropped → clients see timeout.

```python
# Listing backlog size
import socket
srv = socket.socket(); srv.bind(('0.0.0.0',0)); srv.listen(5)
print('listening', srv.getsockname(), 'backlog=5')
```

### Quick check

1. SYN flood targets which part of handshake?
  a. final ACK   b. initial SYN

2. True / False `listen(backlog)` larger number allows more half‑open connections.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 5. Client vs. server socket workflow

| Role | Steps |
|------|-------|
| Client | `socket()` → optional `settimeout()` → `connect()` → `send/recv` → `close()` |
| Server | `socket()` → `bind()` → `listen()` → loop `accept()` → per‑client socket `recv/send` |

Each `accept()` returns a *new* socket object bound to client’s address.

```python
import socket, threading
def handle(csock, addr):
    csock.sendall(b'hi'); csock.close()

srv = socket.socket(); srv.bind(('localhost', 50007)); srv.listen()
threading.Thread(target=lambda: srv.accept() and print('ready')).start()
cl = socket.create_connection(('localhost', 50007)); print(cl.recv(10))
```

### Quick check

1. After `accept()`, server communicates using:
  a. original listening socket   b. newly returned socket

2. True / False `bind()` is required on client sockets.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – OS picks ephemeral port.

</details>